<a href="https://colab.research.google.com/github/ankitarm/SQL_Data_Engineer/blob/main/SQL_FAANG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q1. Meta/Facebook (Hard Level)

---

**Find the famous percentage of each user.**

> *Famous Percentage = number of followers a user has / total number of users on the platform*

---

### 📘 Schema and Dataset

**`famous` Table**

| user\_id | follower\_id |
| -------- | ------------ |
| 1        | 2            |
| 1        | 3            |
| 2        | 4            |
| 5        | 1            |
| 5        | 3            |
| 11       | 7            |
| 12       | 8            |
| 13       | 5            |
| 13       | 10           |
| 14       | 12           |
| 14       | 3            |
| 15       | 14           |
| 15       | 13           |

---

### 🧠 Explanation of the Query

1. **`distinct_users` CTE:**
   Uses `UNION` on `user_id` and `follower_id` to find the total unique users on the platform.

2. **`follower_count` CTE:**
   Groups by `user_id` and counts how many followers each user has.

3. **Final SELECT:**
   Joins the total user count with follower counts to calculate
   `famous_percentage = follower_count / total_users`.


```sql
/*
CREATE TABLE famous (user_id INT, follower_id INT);
INSERT INTO famous VALUES
(1, 2), (1, 3), (2, 4), (5, 1), (5, 3),
(11, 7), (12, 8), (13, 5), (13, 10),
(14, 12), (14, 3), (15, 14), (15, 13);

select * from famous;
*/

WITH USERFOLTBL AS (
  select user_id AS ID from famous
  UNION
  SELECT follower_id AS ID FROM famous)
SELECT user_id,
ROUND((COUNT(DISTINCT follower_id)*1.0/COUNT(DISTINCT ID ))*100,2) AS PERCENTAGE
FROM famous, USERFOLTBL  
GROUP BY user_id



## Q2. LinkedIn (Hard Level)

---

**Find how many users had Google as their next employer immediately after Microsoft (no employers in between).**

---

### 📘 Schema and Dataset

**`linkedin_users` Table**

| user\_id | employer  | position         | start\_date | end\_date  |
| -------- | --------- | ---------------- | ----------- | ---------- |
| 1        | Microsoft | developer        | 2020-04-13  | 2021-11-01 |
| 1        | Google    | developer        | 2021-11-01  | NULL       |
| 2        | Google    | manager          | 2021-01-01  | 2021-01-11 |
| 2        | Microsoft | manager          | 2021-01-11  | NULL       |
| 3        | Microsoft | analyst          | 2019-03-15  | 2020-07-24 |
| 3        | Amazon    | analyst          | 2020-08-01  | 2020-11-01 |
| 3        | Google    | senior analyst   | 2020-11-01  | 2021-03-04 |
| 4        | Google    | junior developer | 2018-06-01  | 2021-11-01 |
| 4        | Google    | senior developer | 2021-11-01  | NULL       |
| 5        | Microsoft | manager          | 2017-09-26  | NULL       |
| 6        | Google    | CEO              | 2015-10-02  | NULL       |

---

### 🧠 Explanation of the Query

1. **CTE with `LEAD()` Function:**
   For each user, sort jobs by `start_date` and use `LEAD()` to fetch the **next employer** and **next start\_date** after the current row.

2. **Filter Microsoft Records:**
   In the CTE, focus on rows where the **current employer is Microsoft** and **next employer is Google**.

3. **Final SELECT:**
   Count how many users had Google immediately after Microsoft (without a job in between).

```

𝐒𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐃𝐚𝐭𝐚𝐬𝐞𝐭:
CREATE TABLE linkedin_users (user_id INT,employer VARCHAR(255),position VARCHAR(255),start_date DATETIME,end_date DATETIME);

INSERT INTO linkedin_users (user_id, employer, position, start_date, end_date) VALUES(1, 'Microsoft', 'developer', '2020-04-13', '2021-11-01'),(1, 'Google', 'developer', '2021-11-01', NULL),(2, 'Google', 'manager', '2021-01-01', '2021-01-11'),(2, 'Microsoft', 'manager', '2021-01-11', NULL),(3, 'Microsoft', 'analyst', '2019-03-15', '2020-07-24'),(3, 'Amazon', 'analyst', '2020-08-01', '2020-11-01'),(3, 'Google', 'senior analyst', '2020-11-01', '2021-03-04'),(4, 'Google', 'junior developer', '2018-06-01', '2021-11-01'),(4, 'Google', 'senior developer', '2021-11-01', NULL),(5, 'Microsoft', 'manager', '2017-09-26', NULL),(6, 'Google', 'CEO', '2015-10-02', NULL);
```
-----------

```sql


WITH NXTEMPTBL AS(
	SELECT user_id,employer,
	LEAD(employer) OVER(PARTITION BY user_id ORDER BY start_date) AS NXTEMP
	FROM linkedin_users)
SELECT user_id, employer,NXTEMP  FROM NXTEMPTBL
WHERE employer = 'Microsoft' AND NXTEMP = 'Google'

Output:

1	Microsoft	Google
```

## Q3. American Express (Medium Level)

---

**Find the customer with the third highest total transaction amount. Output the customer’s id, first name, and last name. Use ranking with no gaps.**

---

### 📘 Schema and Dataset

**`customers` Table**

| id | first\_name | last\_name | city        | address      | phone\_number |
| -- | ----------- | ---------- | ----------- | ------------ | ------------- |
| 1  | Jill        | Doe        | New York    | 123 Main St  | 555-1234      |
| 2  | Henry       | Smith      | Los Angeles | 456 Oak Ave  | 555-5678      |
| 3  | William     | Johnson    | Chicago     | 789 Pine Rd  | 555-8765      |
| 4  | Emma        | Daniel     | Houston     | 321 Maple Dr | 555-4321      |
| 5  | Charlie     | Davis      | Phoenix     | 654 Elm St   | 555-6789      |

**`card_orders` Table**

| order\_id | cust\_id | order\_date         | order\_details | total\_order\_cost |
| --------- | -------- | ------------------- | -------------- | ------------------ |
| 1         | 1        | 2024-11-01 10:00:00 | Electronics    | 200                |
| 2         | 2        | 2024-11-02 11:30:00 | Groceries      | 150                |
| 3         | 1        | 2024-11-03 15:45:00 | Clothing       | 120                |
| 4         | 3        | 2024-11-04 09:10:00 | Books          | 90                 |
| 8         | 3        | 2024-11-08 10:20:00 | Groceries      | 130                |
| 9         | 1        | 2024-11-09 12:00:00 | Books          | 180                |
| 10        | 4        | 2024-11-10 11:15:00 | Electronics    | 200                |
| 11        | 5        | 2024-11-11 14:45:00 | Furniture      | 150                |
| 12        | 2        | 2024-11-12 09:30:00 | Furniture      | 180                |

---

### 🧠 Explanation of the Query

1. **CustomerTransactionTotals CTE:**
   Sum the `total_order_cost` for each `cust_id` from `card_orders`.

2. **RankedTransactions CTE:**
   Use `DENSE_RANK()` to rank customers by their total transaction amount in descending order (no gaps in rank).

3. **Final Selection:**
   Join the 3rd ranked customer with the `customers` table and retrieve their `id`, `first_name`, and `last_name`.

```sql

𝐒𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐃𝐚𝐭𝐚𝐬𝐞𝐭:
CREATE TABLE customers (id INT,first_name VARCHAR(50),last_name VARCHAR(50),city VARCHAR(100),address VARCHAR(200),phone_number VARCHAR(20));

INSERT INTO customers (id, first_name, last_name, city, address, phone_number) VALUES(1, 'Jill', 'Doe', 'New York', '123 Main St', '555-1234'),(2, 'Henry', 'Smith', 'Los Angeles', '456 Oak Ave', '555-5678'),(3, 'William', 'Johnson', 'Chicago', '789 Pine Rd', '555-8765'),(4, 'Emma', 'Daniel', 'Houston', '321 Maple Dr', '555-4321'),(5, 'Charlie', 'Davis', 'Phoenix', '654 Elm St', '555-6789');

CREATE TABLE card_orders (order_id INT,cust_id INT,order_date DATETIME,order_details VARCHAR(255),total_order_cost INT);

INSERT INTO card_orders (order_id, cust_id, order_date, order_details, total_order_cost) VALUES(1, 1, '2024-11-01 10:00:00', 'Electronics', 200),(2, 2, '2024-11-02 11:30:00', 'Groceries', 150),(3, 1, '2024-11-03 15:45:00', 'Clothing', 120),(4, 3, '2024-11-04 09:10:00', 'Books', 90),(8, 3, '2024-11-08 10:20:00', 'Groceries', 130),(9, 1, '2024-11-09 12:00:00', 'Books', 180),(10, 4, '2024-11-10 11:15:00', 'Electronics', 200),(11, 5, '2024-11-11 14:45:00', 'Furniture', 150),(12, 2, '2024-11-12 09:30:00', 'Furniture', 180);
```

```sql

sOLUTION 1:

WITH HIGHTRAN AS (
  SELECT cust_id,SUM(total_order_cost) AS TOTAL_AMOUNT
  FROM card_orders
  GROUP BY cust_id
  ORDER BY TOTAL_AMOUNT DESC
  LIMIT 1 OFFSET 2)
SELECT cust_id, first_name, last_name
FROM HIGHTRAN HT LEFT JOIN  customers C
ON C.id = HT.cust_id


SOLUTION 2:

WITH CustomerTotals AS (
    SELECT
        cust_id,
        SUM(total_order_cost) AS total_amount
    FROM card_orders
    GROUP BY cust_id
),
RankedCustomers AS (
    SELECT
        cust_id,
        total_amount,
        DENSE_RANK() OVER (ORDER BY total_amount DESC) AS rnk
    FROM CustomerTotals
)
SELECT
    c.id AS cust_id,
    c.first_name,
    c.last_name
FROM RankedCustomers r
JOIN customers c ON r.cust_id = c.id
WHERE r.rnk = 3;


| Requirement                              | Use                        |
| ---------------------------------------- | -------------------------- |
| Just the 3rd highest row (ignoring ties) | `LIMIT 1 OFFSET 2`         |
| All customers tied at 3rd highest amount | `RANK()` or `DENSE_RANK()` |
| Need clarity, flexibility, and accuracy  | `RANK()`/`DENSE_RANK()`    |



## Q4. Amazon (Hard Level)

Amazon wants to identify products that are **exclusive to their platform** and not sold on **Top Shop** or **Macy's**.

Two products are considered equal if they share the **same product name** and the **same maximum retail price (mrp column)**.

Your task is to find such exclusive Amazon products and output the following details:

* product name
* brand name
* price
* rating

### 𝐒𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐃𝐚𝐭𝐚𝐬𝐞𝐭 (Table Format)

#### Table: `innerwear_amazon_com`

| product\_name | mrp | price | pdp\_url | brand\_name | product\_category | retailer | description  | rating | review\_count | style\_attributes | total\_sizes | available\_size | color |
| ------------- | --- | ----- | -------- | ----------- | ----------------- | -------- | ------------ | -----: | ------------: | ----------------- | ------------ | --------------- | ----- |
| ProductA      | 100 | 80    | url1     | BrandA      | Category1         | Amazon   | DescriptionA |    4.5 |           100 | StyleA            | M,L          | M               | Red   |
| ProductB      | 200 | 180   | url2     | BrandB      | Category1         | Amazon   | DescriptionB |    4.2 |           150 | StyleB            | S,M,L        | S               | Blue  |
| ProductC      | 300 | 250   | url3     | BrandC      | Category2         | Amazon   | DescriptionC |    4.8 |           200 | StyleC            | L,XL         | L               | Green |

#### Table: `innerwear_macys_com`

| product\_name | mrp | price | pdp\_url | brand\_name | product\_category | retailer | description  | rating | review\_count | style\_attributes | total\_sizes | available\_size | color  |
| ------------- | --- | ----- | -------- | ----------- | ----------------- | -------- | ------------ | -----: | ------------: | ----------------- | ------------ | --------------- | ------ |
| ProductA      | 100 | 85    | url4     | BrandA      | Category1         | Macys    | DescriptionA |    4.5 |            90 | StyleA            | M,L          | M               | Red    |
| ProductD      | 150 | 130   | url5     | BrandD      | Category3         | Macys    | DescriptionD |    4.0 |            80 | StyleD            | S,M          | S               | Yellow |
| ProductE      | 250 | 210   | url6     | BrandE      | Category4         | Macys    | DescriptionE |    3.9 |            60 | StyleE            | M,L          | L               | Black  |

#### Table: `innerwear_topshop_com`

| product\_name | mrp | price | pdp\_url | brand\_name | product\_category | retailer | description  | rating | review\_count | style\_attributes | total\_sizes | available\_size | color  |
| ------------- | --- | ----- | -------- | ----------- | ----------------- | -------- | ------------ | -----: | ------------: | ----------------- | ------------ | --------------- | ------ |
| ProductB      | 200 | 190   | url7     | BrandB      | Category1         | TopShop  | DescriptionB |    4.1 |            95 | StyleB            | S,M,L        | M               | Blue   |
| ProductF      | 100 | 90    | url8     | BrandF      | Category3         | TopShop  | DescriptionF |    3.5 |            50 | StyleF            | XS,S         | S               | Pink   |
| ProductG      | 300 | 270   | url9     | BrandG      | Category5         | TopShop  | DescriptionG |    4.3 |            70 | StyleG            | M,L,XL       | M               | Purple |

---

### 𝐄𝐱𝐩𝐥𝐚𝐧𝐚𝐭𝐢𝐨𝐧 𝐭𝐨 𝐒𝐨𝐥𝐯𝐞 𝐐𝐮𝐞𝐫𝐲

1. **LEFT JOINs**: Join Amazon’s table to Macy’s and TopShop on **both** `product_name` **and** `mrp` to treat products with the same name and MRP as identical across retailers.
2. **Filter Exclusives**: Keep only rows where **no match** is found in Macy’s **and** TopShop (i.e., joined columns are `NULL`), meaning the product exists only on Amazon.
3. **Select & Sort**: Return `product_name`, `brand_name`, `price`, `rating` from Amazon for those exclusive items (optionally sort by `product_name`).

---

**Schema and Dataset:**

```sql
CREATE TABLE innerwear_amazon_com (
  product_name VARCHAR(255),
  mrp VARCHAR(50),
  price VARCHAR(50),
  pdp_url VARCHAR(255),
  brand_name VARCHAR(100),
  product_category VARCHAR(100),
  retailer VARCHAR(100),
  description VARCHAR(255),
  rating FLOAT,
  review_count INT,
  style_attributes VARCHAR(255),
  total_sizes VARCHAR(50),
  available_size VARCHAR(50),
  color VARCHAR(50)
);

CREATE TABLE innerwear_macys_com (
  product_name VARCHAR(255),
  mrp VARCHAR(50),
  price VARCHAR(50),
  pdp_url VARCHAR(255),
  brand_name VARCHAR(100),
  product_category VARCHAR(100),
  retailer VARCHAR(100),
  description VARCHAR(255),
  rating FLOAT,
  review_count FLOAT,
  style_attributes VARCHAR(255),
  total_sizes VARCHAR(50),
  available_size VARCHAR(50),
  color VARCHAR(50)
);

CREATE TABLE innerwear_topshop_com (
  product_name VARCHAR(255),
  mrp VARCHAR(50),
  price VARCHAR(50),
  pdp_url VARCHAR(255),
  brand_name VARCHAR(100),
  product_category VARCHAR(100),
  retailer VARCHAR(100),
  description VARCHAR(255),
  rating FLOAT,
  review_count FLOAT,
  style_attributes VARCHAR(255),
  total_sizes VARCHAR(50),
  available_size VARCHAR(50),
  color VARCHAR(50)
);


INSERT INTO innerwear_topshop_com VALUES
('ProductB', '200', '190', 'url7', 'BrandB', 'Category1', 'TopShop', 'DescriptionB', 4.1, 95, 'StyleB', 'S,M,L', 'M', 'Blue'),
('ProductF', '100', '90', 'url8', 'BrandF', 'Category3', 'TopShop', 'DescriptionF', 3.5, 50, 'StyleF', 'XS,S', 'S', 'Pink'),
('ProductG', '300', '270', 'url9', 'BrandG', 'Category5', 'TopShop', 'DescriptionG', 4.3, 70, 'StyleG', 'M,L,XL', 'M', 'Purple');

INSERT INTO innerwear_amazon_com VALUES
('ProductA', '100', '80', 'url1', 'BrandA', 'Category1', 'Amazon', 'DescriptionA', 4.5, 100, 'StyleA', 'M,L', 'M', 'Red'),
('ProductB', '200', '180', 'url2', 'BrandB', 'Category1', 'Amazon', 'DescriptionB', 4.2, 150, 'StyleB', 'S,M,L', 'S', 'Blue'),
('ProductC', '300', '250', 'url3', 'BrandC', 'Category2', 'Amazon', 'DescriptionC', 4.8, 200, 'StyleC', 'L,XL', 'L', 'Green');

INSERT INTO innerwear_macys_com VALUES
('ProductA', '100', '85', 'url4', 'BrandA', 'Category1', 'Macys', 'DescriptionA', 4.5, 90, 'StyleA', 'M,L', 'M', 'Red'),
('ProductD', '150', '130', 'url5', 'BrandD', 'Category3', 'Macys', 'DescriptionD', 4.0, 80, 'StyleD', 'S,M', 'S', 'Yellow'),
('ProductE', '250', '210', 'url6', 'BrandE', 'Category4', 'Macys', 'DescriptionE', 3.9, 60, 'StyleE', 'M,L', 'L', 'Black');
```

---
```sql
WITH CTE AS (SELECT A.product_name, A.brand_name, A.price, A.rating, A.mrp
FROM innerwear_amazon_com A
LEFT JOIN innerwear_topshop_com T ON A.product_name = T.product_name AND A.mrp = T.mrp
WHERE T.product_name IS NULL)
SELECT A.product_name, A.brand_name, A.price, A.rating
FROM CTE A
LEFT JOIN innerwear_macys_com M ON A.product_name = M.product_name AND A.mrp = M.mrp
WHERE M.product_name IS NULL;




# ✅ Q5. Oracle (Hard Level)

### **Problem Statement**

Write a query that compares each employee's salary to their **manager's salary** and the **average department salary** (excluding the manager’s salary).

Display:

* department
* employee ID
* employee’s salary
* manager’s salary
* department average salary

Order the results by **department** and then by **employee salary (highest → lowest)**.

🔍 By solving this, you’ll learn **CTEs, multiple joins, and GROUP BY**.

---

## 🔹 Google Colab Implementation
```sql
CREATE TABLE employee_o (
    id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    age INT,
    gender VARCHAR(10),
    employee_title VARCHAR(50),
    department VARCHAR(50),
    salary INT,
    manager_id INT
);

INSERT INTO employee_o VALUES
(1, 'Alice', 'Smith', 45, 'F', 'Manager', 'HR', 9000, 1),
(2, 'Bob', 'Johnson', 34, 'M', 'Assistant', 'HR', 4500, 1),
(3, 'Charlie', 'Williams', 28, 'M', 'Coordinator', 'HR', 4800, 1),
(4, 'Diana', 'Brown', 32, 'F', 'Manager', 'IT', 12000, 4),
(5, 'Eve', 'Jones', 27, 'F', 'Analyst', 'IT', 7000, 4),
(6, 'Frank', 'Garcia', 29, 'M', 'Developer', 'IT', 7500, 4),
(7, 'Grace', 'Miller', 30, 'F', 'Manager', 'Finance', 10000, 7),
(8, 'Hank', 'Davis', 26, 'M', 'Analyst', 'Finance', 6200, 7),
(9, 'Ivy', 'Martinez', 31, 'F', 'Clerk', 'Finance', 5900, 7),
(10, 'John', 'Lopez', 36, 'M', 'Manager', 'Marketing', 11000, 10),
(11, 'Kim', 'Gonzales', 29, 'F', 'Specialist', 'Marketing', 6800, 10),
(12, 'Leo', 'Wilson', 27, 'M', 'Coordinator', 'Marketing', 6600, 10);


# Step 5: Query using CTEs

```sql
WITH DepartmentAvgSalary AS (
    SELECT department,
           AVG(salary) AS dept_avg_salary
    FROM employee_o e
    WHERE id != manager_id   -- exclude manager's own salary
    GROUP BY department
),
ManagerSalary AS (
    SELECT e.id AS emp_id,
           e.department,
           e.salary AS emp_salary,
           m.salary AS manager_salary
    FROM employee_o e
    JOIN employee_o m
      ON e.manager_id = m.id
    WHERE e.id != e.manager_id   -- exclude managers who report to themselves
)
SELECT m.department,
       m.emp_id,
       m.emp_salary,
       m.manager_salary,
       d.dept_avg_salary
FROM ManagerSalary m
JOIN DepartmentAvgSalary d
  ON m.department = d.department
ORDER BY m.department, m.emp_salary DESC;
```
---


## 🔹 Explanation of Query Steps

1. **`DepartmentAvgSalary` CTE**

   * Computes the average salary for each department **excluding the manager’s own salary** (`id != manager_id`).

2. **`ManagerSalary` CTE**

   * Self-join on `manager_id` to fetch each employee’s manager’s salary.
   * Excludes managers who report to themselves.

3. **Final Select**

   * Joins both CTEs to bring **employee’s salary, manager’s salary, and department average salary** together.
   * Ordered by department, then salary (highest → lowest).

---


##Q6. Walmart, Paypal (Medium Level)

Find managers with at least 7 direct reporting employees. In situations where user is reporting to himself/herself, count that also.
Output first names of managers.

🔍By solving this, you'll learn how to use self join. Give it a try and share the output! 👇

𝐒𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐃𝐚𝐭𝐚𝐬𝐞𝐭:
CREATE TABLE employees (id INT PRIMARY KEY,first_name VARCHAR(50),last_name VARCHAR(50),age INT,sex VARCHAR(10),employee_title VARCHAR(50),department VARCHAR(50),salary INT,target INT,bonus INT,email VARCHAR(100),city VARCHAR(50),address VARCHAR(255),manager_id INT);

INSERT INTO employees (id, first_name, last_name, age, sex, employee_title, department, salary, target, bonus, email, city, address, manager_id) VALUES(1, 'Alice', 'Smith', 40, 'F', 'Manager', 'Sales', 90000, 100000, 15000, 'alice.smith@example.com', 'New York', '123 Main St', 1),(2, 'Bob', 'Johnson', 35, 'M', 'Team Lead', 'Sales', 80000, 95000, 12000, 'bob.johnson@example.com', 'Chicago', '456 Oak St', 1),(3, 'Carol', 'Williams', 30, 'F', 'Sales Executive', 'Sales', 70000, 85000, 10000, 'carol.williams@example.com', 'New York', '789 Pine St', 1),(4, 'David', 'Brown', 28, 'M', 'Sales Executive', 'Sales', 68000, 80000, 9000, 'david.brown@example.com', 'Chicago', '101 Maple St', 1),(5, 'Emma', 'Jones', 32, 'F', 'Sales Executive', 'Sales', 71000, 86000, 9500, 'emma.jones@example.com', 'New York', '202 Cedar St', 1),(6, 'Frank', 'Miller', 45, 'M', 'Manager', 'Engineering', 95000, 105000, 16000, 'frank.miller@example.com', 'San Francisco', '303 Spruce St', 6),(7, 'Grace', 'Davis', 29, 'F', 'Engineer', 'Engineering', 73000, 87000, 11000, 'grace.davis@example.com', 'San Francisco', '404 Willow St', 6);
-----------

I have provided an explanation and query, but I encourage you to try solving it first. Later, you can check the query for reference.

𝐄𝐱𝐩𝐥𝐚𝐧𝐚𝐭𝐢𝐨𝐧 𝐭𝐨 𝐒𝐨𝐥𝐯𝐞 𝐐𝐮𝐞𝐫𝐲
1. Self-reporting Employees: Since manager_id points back to the id column in the same table, cases where manager_id = id (self-reporting) are automatically included.

2. JOIN: The query joins the employees table with itself to match employees (e) with their managers (m) based on manager_id.

3. COUNT and HAVING: After grouping by the manager’s id, HAVING COUNT(e.id) >= 7 filters for managers with at least 7 direct reports.

##Q7. Microsoft (Hard Level)

Find the total number of downloads for paying and non-paying users by date. Include only records where non-paying customers have more downloads than paying customers. The output should be sorted by earliest date first and contain 3 columns date, non-paying downloads, paying downloads.

Note: In Oracle you should use "date" when referring to date column (reserved keyword).

🔍By solving this, you'll learn how to use join, groupby and having. Give it a try and share the output!👇

𝐒𝐜𝐡𝐞𝐦𝐚 𝐚𝐧𝐝 𝐃𝐚𝐭𝐚𝐬𝐞𝐭:
CREATE TABLE ms_user_dimension (user_id INT PRIMARY KEY,acc_id INT);
INSERT INTO ms_user_dimension (user_id, acc_id) VALUES (1, 101),(2, 102),(3, 103),(4, 104),(5, 105);

CREATE TABLE ms_acc_dimension (acc_id INT PRIMARY KEY,paying_customer VARCHAR(10));
INSERT INTO ms_acc_dimension (acc_id, paying_customer) VALUES (101, 'Yes'),(102, 'No'),(103, 'Yes'),(104, 'No'),(105, 'No');

CREATE TABLE ms_download_facts (date DATETIME,user_id INT,downloads INT);
INSERT INTO ms_download_facts (date, user_id, downloads) VALUES ('2024-10-01', 1, 10),('2024-10-01', 2, 15),('2024-10-02', 1, 8),('2024-10-02', 3, 12),('2024-10-02', 4, 20),('2024-10-03', 2, 25),('2024-10-03', 5, 18);
-----------

I have provided an explanation and query, but I encourage you to try solving it first. Later, you can check the query for reference.

𝐄𝐱𝐩𝐥𝐚𝐧𝐚𝐭𝐢𝐨𝐧 𝐭𝐨 𝐒𝐨𝐥𝐯𝐞 𝐐𝐮𝐞𝐫𝐲
1. CONVERT(DATE, d.date): We convert the datetime to date to ensure we’re aggregating by date only.

2. SUM with CASE: The CASE within SUM helps in counting downloads conditionally:
For non-paying customers (a.paying_customer = 'No'), it sums downloads only when the customer is non-paying.
For paying customers (a.paying_customer = 'Yes'), it sums downloads only when the customer is paying.

3. HAVING Clause: Filters results to include only dates where non-paying customers have more downloads than paying customers.

4. ORDER BY [date] ASC: Orders the output by the date in ascending order, as required.



##Q8. Apple | Microsoft | Dell (Easy Level)

---

**Write a query that returns the number of unique users per client per month.**
---

### Sample Data: `fact_events`

| id | time\_id   | user\_id   | customer\_id | client\_id | event\_type         | event\_id |
| -- | ---------- | ---------- | ------------ | ---------- | ------------------- | --------- |
| 1  | 2020-02-28 | 3668-QPYBK | Sendit       | desktop    | message sent        | 3         |
| 2  | 2020-02-28 | 7892-POOKP | Connectix    | mobile     | file received       | 2         |
| 3  | 2020-04-03 | 9763-GRSKD | Zoomit       | desktop    | video call received | 7         |
| 4  | 2020-04-02 | 9763-GRSKD | Connectix    | desktop    | video call received | 7         |
| 5  | 2020-02-06 | 9237-HQITU | Sendit       | desktop    | video call received | 7         |
| 6  | 2020-02-27 | 8191-XWSZG | Connectix    | desktop    | file received       | 2         |
| 7  | 2020-04-03 | 9237-HQITU | Connectix    | desktop    | video call received | 7         |
| 8  | 2020-03-01 | 9237-HQITU | Connectix    | mobile     | message received    | 4         |
| 9  | 2020-04-02 | 4190-MFLUW | Connectix    | mobile     | video call received | 7         |
| 10 | 2020-04-21 | 9763-GRSKD | Sendit       | desktop    | file received       | 2         |

---

### How to Think About the Solution

1. **Extract month & year**
   Use `FORMAT(time_id, 'yyyy-MM')` to get the month and year for grouping.

2. **Count distinct users**
   Use `COUNT(DISTINCT user_id)` to find unique users per group.

3. **Group by client and month**
   Group by both `client_id` and the formatted date.

4. **Order for clarity**
   Sort the output by `client_id` and month to make it readable.

---






## Q9. Walmart (Hard Level)

---

**Identify users who started a session and placed an order on the same day. For these users, calculate the total number of orders and the total order value for that day. Your output should include the user, the session date, the total number of orders, and the total order value for that day.**

---

### Sample Data: `sessions`, `order_summary`

#### `sessions`

| session\_id | user\_id | session\_date       |
| ----------- | -------- | ------------------- |
| 1           | 1        | 2024-01-01 00:00:00 |
| 2           | 2        | 2024-01-02 00:00:00 |
| 3           | 3        | 2024-01-05 00:00:00 |
| 4           | 3        | 2024-01-05 00:00:00 |
| 5           | 4        | 2024-01-03 00:00:00 |
| 6           | 4        | 2024-01-03 00:00:00 |
| 7           | 5        | 2024-01-04 00:00:00 |
| 8           | 5        | 2024-01-04 00:00:00 |
| 9           | 3        | 2024-01-05 00:00:00 |
| 10          | 5        | 2024-01-04 00:00:00 |

#### `order_summary`

| order\_id | user\_id | order\_value | order\_date         |
| --------- | -------- | ------------ | ------------------- |
| 1         | 1        | 152          | 2024-01-01 00:00:00 |
| 2         | 2        | 485          | 2024-01-02 00:00:00 |
| 3         | 3        | 398          | 2024-01-05 00:00:00 |
| 4         | 3        | 320          | 2024-01-05 00:00:00 |
| 5         | 4        | 156          | 2024-01-03 00:00:00 |
| 6         | 4        | 121          | 2024-01-03 00:00:00 |
| 7         | 5        | 238          | 2024-01-04 00:00:00 |
| 8         | 5        | 70           | 2024-01-04 00:00:00 |
| 9         | 3        | 152          | 2024-01-05 00:00:00 |
| 10        | 5        | 171          | 2024-01-04 00:00:00 |

---

### How to Think About the Solution

1. **Join tables**
   Join `sessions` and `order_summary` on `user_id`.

2. **Match session and order dates**
   Use `CONVERT(DATE, session_date)` and `CONVERT(DATE, order_date)` to ignore time and match by calendar day.

3. **Aggregate**
   Use `COUNT(order_id)` for number of orders, and `SUM(order_value)` for total value.

4. **Group by**
   Group by `user_id` and session date.

5. **Having**
   Filter to only include users who placed at least 1 order that day using `HAVING COUNT(order_id) > 0`.

---


## Q10. Amazon | DoorDash (Medium Level)

---

**Write a query to find the job title(s) of the highest-paid employee(s). Return all titles that correspond to the highest salary in the company.**

---

### Sample Data: `worker`, `title`

#### `worker`

| worker\_id | first\_name | last\_name | salary | joining\_date | department  |
| ---------- | ----------- | ---------- | ------ | ------------- | ----------- |
| 1          | John        | Doe        | 80000  | 2020-01-15    | Engineering |
| 2          | Jane        | Smith      | 120000 | 2019-03-10    | Marketing   |
| 3          | Alice       | Brown      | 120000 | 2021-06-21    | Sales       |
| 4          | Bob         | Davis      | 75000  | 2018-04-30    | Engineering |
| 5          | Charlie     | Miller     | 95000  | 2021-01-15    | Sales       |

#### `title`

| worker\_ref\_id | worker\_title      | affected\_from |
| --------------- | ------------------ | -------------- |
| 1               | Engineer           | 2020-01-15     |
| 2               | Marketing Manager  | 2019-03-10     |
| 3               | Sales Manager      | 2021-06-21     |
| 4               | Junior Engineer    | 2018-04-30     |
| 5               | Senior Salesperson | 2021-01-15     |

---

### How to Think About the Solution

1. **Find the max salary**
   Use a subquery like `(SELECT MAX(salary) FROM worker)` to get the highest salary.

2. **Join tables**
   Join `worker` and `title` using `worker.worker_id = title.worker_ref_id` to associate each worker with their title.

3. **Filter**
   Use `WHERE salary = (SELECT MAX(salary) ...)` to select only the highest-paid worker(s).

4. **Select title**
   Return the `worker_title` of the filtered records.

---


## Q11. Uber | Medium Level

---

**Write a query to find the top 3 business purpose categories that generate the most miles driven. Only include rides where the category is 'Business'.**

---

### Sample Data: `my_uber_drives`

| start\_date      | end\_date        | category | start           | stop            | miles | purpose         |
| ---------------- | ---------------- | -------- | --------------- | --------------- | ----- | --------------- |
| 2016-01-01 21:11 | 2016-01-01 21:17 | Business | Fort Pierce     | Fort Pierce     | 5.1   | Meal/Entertain  |
| 2016-01-02 01:25 | 2016-01-02 01:37 | Business | Fort Pierce     | Fort Pierce     | 5     | NULL            |
| 2016-01-02 20:25 | 2016-01-02 20:38 | Business | Fort Pierce     | Fort Pierce     | 4.8   | Errand/Supplies |
| 2016-01-05 17:31 | 2016-01-05 17:45 | Business | Fort Pierce     | Fort Pierce     | 4.7   | Meeting         |
| 2016-01-06 14:42 | 2016-01-06 15:49 | Business | Fort Pierce     | West Palm Beach | 63.7  | Customer Visit  |
| 2016-01-06 17:15 | 2016-01-06 17:19 | Business | West Palm Beach | West Palm Beach | 4.3   | Meal/Entertain  |
| 2016-01-06 17:30 | 2016-01-06 17:35 | Business | West Palm Beach | Palm Beach      | 7.1   | Meeting         |

---

### How to Think About the Solution

1. **Filter on category**
   Use `WHERE category = 'Business'` to focus only on business trips.

2. **Group by purpose**
   Aggregate total miles using `SUM(miles)` grouped by each `purpose`.

3. **Order results**
   Use `ORDER BY total_miles DESC` to rank by mileage.

4. **Limit to Top 3**
   Use `LIMIT 3` to return the top 3 business purpose categories.

---



## Q12. JP Morgan | Chase | Bloomberg (Medium Level)

---

**Write a query to find all invalid transactions in December 2022. A transaction is considered invalid if it occurs outside business hours (Mon–Fri, 09:00–16:00), on weekends, or on Irish public holidays (25th and 26th December). Return the `transaction_id` of such records.**

---

### Sample Data: `boi_transactions`

| transaction\_id | time\_stamp      |
| --------------- | ---------------- |
| 1051            | 2022-12-03 10:15 |
| 1052            | 2022-12-03 17:00 |
| 1053            | 2022-12-04 10:00 |
| 1054            | 2022-12-04 14:00 |
| 1055            | 2022-12-05 08:59 |
| 1056            | 2022-12-05 16:01 |
| 1057            | 2022-12-06 09:00 |
| 1058            | 2022-12-06 15:59 |
| 1059            | 2022-12-07 12:00 |
| 1060            | 2022-12-08 09:00 |
| 1061            | 2022-12-09 10:00 |
| 1062            | 2022-12-10 11:00 |
| 1063            | 2022-12-10 17:30 |
| 1064            | 2022-12-11 12:00 |
| 1065            | 2022-12-12 08:59 |
| 1066            | 2022-12-12 16:01 |
| 1067            | 2022-12-25 10:00 |
| 1068            | 2022-12-25 15:00 |
| 1069            | 2022-12-26 09:00 |
| 1070            | 2022-12-26 14:00 |
| 1071            | 2022-12-26 16:30 |
| 1072            | 2022-12-27 09:00 |
| 1073            | 2022-12-28 08:30 |
| 1074            | 2022-12-29 16:15 |
| 1075            | 2022-12-30 14:00 |
| 1076            | 2022-12-31 10:00 |

---

### How to Think About the Solution

1. **Filter December 2022**
   Use `MONTH(time_stamp) = 12 AND YEAR(time_stamp) = 2022` to isolate December transactions.

2. **Check for weekends**
   Use `DATEPART(WEEKDAY, time_stamp)` to identify Sundays and Saturdays.

3. **Check for time range**
   Use `CAST(time_stamp AS TIME) < '09:00:00' OR > '16:00:00'` to flag anything outside business hours.

4. **Check for holidays**
   Add an extra condition for `DAY(time_stamp) IN (25, 26)` to filter Irish public holidays.

---




## Q13. Google (Medium Level)

---

**Find all records from days when the number of distinct users receiving emails was greater than the number of distinct users sending emails.**

---

### Sample Data: `google_gmail_emails`

| id | from\_user         | to\_user           | day |
| -- | ------------------ | ------------------ | --- |
| 0  | 6edf0be4b2267df1fa | 75d295377a46f83236 | 10  |
| 1  | 6edf0be4b2267df1fa | 32ded68d89443e808  | 6   |
| 2  | 6edf0be4b2267df1fa | 55e60cfcc9dc49c17e | 10  |
| 3  | 6edf0be4b2267df1fa | e0e0defbb9ec47f6f7 | 6   |
| 4  | 6edf0be4b2267df1fa | 47be2887786891367e | 1   |
| 5  | 6edf0be4b2267df1fa | 2813e59cf6c1ff698e | 6   |
| 6  | 6edf0be4b2267df1fa | a84065b7933ad01019 | 8   |
| 7  | 6edf0be4b2267df1fa | 850badf89ed8f06854 | 1   |
| 8  | 6edf0be4b2267df1fa | 6b503743a13d778200 | 1   |
| 9  | 6edf0be4b2267df1fa | d63386c884aeb9f71d | 3   |
| 10 | 6edf0be4b2267df1fa | 5b8754928306a18b68 | 2   |
| 11 | 6edf0be4b2267df1fa | 6edf0be4b2267df1fa | 8   |
| 12 | 6edf0be4b2267df1fa | 406539987dd9b679c0 | 9   |
| 13 | 6edf0be4b2267df1fa | 114bafadff2d882864 | 5   |
| 14 | 6edf0be4b2267df1fa | 157e3e9278e32aba3e | 2   |
| 15 | 75d295377a46f83236 | 75d295377a46f83236 | 6   |
| 16 | 75d295377a46f83236 | d63386c884aeb9f71d | 8   |
| 17 | 75d295377a46f83236 | 55e60cfcc9dc49c17e | 3   |
| 18 | 75d295377a46f83236 | 47be2887786891367e | 10  |
| 19 | 75d295377a46f83236 | 5b8754928306a18b68 | 10  |
| 20 | 75d295377a46f83236 | 850badf89ed8f06854 | 7   |

---

### How to Think About the Solution

1. **Calculate distinct counts per day**
   Use a CTE or subqueries to find the number of distinct `to_user` (receivers) and `from_user` (senders) per `day`.

2. **Compare distinct counts**
   Select days where the number of distinct receivers is greater than distinct senders.

3. **Join back to original**
   Retrieve all records from those filtered days by joining on the `day` field.

---



## Q14. Amazon | Salesforce (Basic Level)

---

**What is the total sales revenue of Samantha and Lisa?**

---

### Sample Data: `sales_performance`

| salesperson | widget\_sales | sales\_revenue | id |
| ----------- | ------------- | -------------- | -- |
| Jim         | 810           | 40500          | 1  |
| Bobby       | 661           | 33050          | 2  |
| Samantha    | 1006          | 50300          | 3  |
| Taylor      | 984           | 49200          | 4  |
| Tom         | 403           | 20150          | 5  |
| Pat         | 715           | 35750          | 6  |
| Lisa        | 1247          | 62350          | 7  |

---

### How to Think About the Solution

1. **Filter for Samantha and Lisa**
   Use `WHERE salesperson IN ('Samantha', 'Lisa')` to target these two salespeople.

2. **Sum sales revenue**
   Use `SUM(sales_revenue)` to get their combined sales revenue.

---




## Q15. Expedia | Airbnb (Basic Level)

---

**Find the number of rows for each review score earned by 'Hotel Arena'. Output the hotel name (which should be 'Hotel Arena'), review score, and the count of rows with that score for the specified hotel.**

---

### Sample Data: `hotel_reviews`

| hotel\_address | additional\_number\_of\_scoring | review\_date | average\_score | hotel\_name | reviewer\_nationality | negative\_review | review\_total\_negative\_word\_counts | total\_number\_of\_reviews | positive\_review | review\_total\_positive\_word\_counts | total\_number\_of\_reviews\_reviewer\_has\_given | reviewer\_score | tags          | days\_since\_review | lat     | lng      |
| -------------- | ------------------------------- | ------------ | -------------- | ----------- | --------------------- | ---------------- | ------------------------------------- | -------------------------- | ---------------- | ------------------------------------- | ------------------------------------------------ | --------------- | ------------- | ------------------- | ------- | -------- |
| 123 Main St    | 5                               | 2024-01-01   | 8.5            | Hotel Arena | American              | Noisy room       | 3                                     | 200                        | Great staff      | 5                                     | 10                                               | 8.0             | Family stay   | 100 days            | 40.7128 | -74.0060 |
| 123 Main St    | 2                               | 2024-01-02   | 8.5            | Hotel Arena | British               | Small bathroom   | 2                                     | 200                        | Clean room       | 6                                     | 5                                                | 9.0             | Solo traveler | 95 days             | 40.7128 | -74.0060 |
| 123 Main St    | 3                               | 2024-01-03   | 8.5            | Hotel Arena | Canadian              | Slow service     | 4                                     | 200                        | Nice view        | 7                                     | 3                                                | 6.0             | Couple stay   | 90 days             | 40.7128 | -74.0060 |

---

### How to Think About the Solution

1. **Filter by hotel name**
   Use `WHERE hotel_name = 'Hotel Arena'` to limit rows to that hotel.

2. **Group by reviewer\_score**
   Group the filtered rows by `reviewer_score`.

3. **Count rows per score**
   Use `COUNT(*)` to find how many reviews correspond to each score.

4. **Select hotel name, score, and count**
   Output the `hotel_name`, `reviewer_score`, and the count of rows per score.

---



## Q16. LinkedIn | Dropbox (Basic Level)

---

**Write a query that calculates the absolute difference between the highest salaries found in the marketing and engineering departments. Output just the absolute difference in salaries.**

---

### Sample Data: `db_employee`

| id    | first\_name | last\_name  | salary | department\_id |
| ----- | ----------- | ----------- | ------ | -------------- |
| 10306 | Ashley      | Li          | 28516  | 4              |
| 10307 | Joseph      | Solomon     | 19945  | 1              |
| 10311 | Melissa     | Holmes      | 33575  | 1              |
| 10316 | Beth        | Torres      | 34902  | 1              |
| 10317 | Pamela      | Rodriguez   | 48187  | 4              |
| 10320 | Gregory     | Cook        | 22681  | 4              |
| 10324 | William     | Brewer      | 15947  | 1              |
| 10329 | Christopher | Ramos       | 37710  | 4              |
| 10333 | Jennifer    | Blankenship | 13433  | 4              |
| 10339 | Robert      | Mills       | 13188  | 1              |

---

### Sample Data: `db_dept`

| id | department     |
| -- | -------------- |
| 1  | engineering    |
| 2  | human resource |
| 3  | operation      |
| 4  | marketing      |

---

### How to Think About the Solution

1. **Use CASE**
   Select salaries based on department, using a CASE statement to separate marketing and engineering salaries.

2. **Find MAX salaries**
   Use `MAX()` to get the highest salary for each department.

3. **Calculate absolute difference**
   Use `ABS()` function to get the absolute difference between the two highest salaries.

---




## Q17. Nvidia | Microsoft (Medium Level)

---

**Find the number of transactions that occurred for each product. Output the product name along with the corresponding number of transactions and order records by the product id in ascending order. You can ignore products without transactions.**

---

### Sample Data: `excel_sql_inventory_data`

| product\_id | product\_name | product\_type | unit | price\_unit | wholesale | current\_inventory |
| ----------- | ------------- | ------------- | ---- | ----------- | --------- | ------------------ |
| 1           | strawberry    | produce       | lb   | 3.28        | 1.77      | 13                 |
| 2           | apple\_fuji   | produce       | lb   | 1.44        | 0.43      | 2                  |
| 3           | orange        | produce       | lb   | 1.02        | 0.37      | 2                  |
| 4           | clementines   | produce       | lb   | 1.19        | 0.44      | 44                 |
| 5           | blood\_orange | produce       | lb   | 3.86        | 1.66      | 19                 |

---

### Sample Data: `excel_sql_transaction_data`

| transaction\_id | time                | product\_id |
| --------------- | ------------------- | ----------- |
| 153             | 2016-01-06 08:57:52 | 1           |
| 91              | 2016-01-07 12:17:27 | 1           |
| 31              | 2016-01-05 13:19:25 | 1           |
| 24              | 2016-01-03 10:47:44 | 3           |
| 4               | 2016-01-06 17:57:42 | 3           |
| 163             | 2016-01-03 10:11:22 | 3           |
| 92              | 2016-01-08 12:03:20 | 2           |
| 32              | 2016-01-04 19:37:14 | 4           |
| 253             | 2016-01-06 14:15:20 | 5           |
| 118             | 2016-01-06 14:27:33 | 5           |

---

### How to Think About the Solution

1. **Join the tables**
   Perform an INNER JOIN between `excel_sql_inventory_data` (alias `inv`) and `excel_sql_transaction_data` (alias `trans`) on `product_id`. This excludes products without transactions.

2. **Count transactions**
   Use `COUNT(trans.transaction_id)` to find the number of transactions per product.

3. **Group and order**
   GROUP BY `inv.product_id`, `inv.product_name` to get counts per product, and ORDER BY `inv.product_id` ASC.

---


## Q18. Amazon (Medium Level)

---

**Write a query that'll identify returning active users. A returning active user is a user that has made a second purchase within 7 days of any other of their purchases. Output a list of user\_ids of these returning active users.**

---

### Sample Data: `amazon_transactions`

| id | user\_id | item    | created\_at         | revenue |
| -- | -------- | ------- | ------------------- | ------- |
| 1  | 109      | milk    | 2020-03-03 00:00:00 | 123     |
| 2  | 139      | biscuit | 2020-03-18 00:00:00 | 421     |
| 3  | 120      | milk    | 2020-03-18 00:00:00 | 176     |
| 4  | 108      | banana  | 2020-03-18 00:00:00 | 862     |
| 5  | 130      | milk    | 2020-03-28 00:00:00 | 333     |
| 6  | 103      | bread   | 2020-03-29 00:00:00 | 862     |
| 7  | 122      | banana  | 2020-03-07 00:00:00 | 952     |
| 8  | 125      | bread   | 2020-03-13 00:00:00 | 317     |
| 9  | 139      | bread   | 2020-03-30 00:00:00 | 929     |
| 10 | 141      | banana  | 2020-03-17 00:00:00 | 812     |
| 11 | 116      | bread   | 2020-03-31 00:00:00 | 226     |
| 12 | 128      | bread   | 2020-03-04 00:00:00 | 112     |
| 13 | 146      | biscuit | 2020-03-04 00:00:00 | 362     |
| 14 | 119      | banana  | 2020-03-28 00:00:00 | 127     |
| 15 | 142      | bread   | 2020-03-09 00:00:00 | 503     |
| 16 | 122      | bread   | 2020-03-06 00:00:00 | 593     |
| 17 | 128      | biscuit | 2020-03-24 00:00:00 | 160     |
| 18 | 112      | banana  | 2020-03-24 00:00:00 | 262     |
| 19 | 149      | banana  | 2020-03-29 00:00:00 | 382     |
| 20 | 100      | banana  | 2020-03-18 00:00:00 | 599     |

---

### How to Think About the Solution

1. **Self-Join:**
   Join the table with itself on `user_id` to compare each purchase against others by the same user.

2. **Conditions:**

   * `a.user_id = b.user_id` ensures the same user.
   * `a.created_at < b.created_at` ensures the second purchase is after the first.
   * `DATEDIFF(day, a.created_at, b.created_at) <= 7` filters for a second purchase within 7 days.

3. **Output:**
   Use `DISTINCT` to return unique user\_ids of those who made a qualifying second purchase.

---



## Q19. Netflix (Hard Level)

---

**Find the genre of the person with the most number of Oscar winnings.**
If multiple people have the same highest number of wins, return the one whose name is first alphabetically. Use the names as keys when joining tables.

---

### Sample Data

**nominee\_information**

| name              | amg\_person\_id | top\_genre | birthday   | id  |
| ----------------- | --------------- | ---------- | ---------- | --- |
| Jennifer Lawrence | P562566         | Drama      | 1990-08-15 | 755 |
| Jonah Hill        | P418718         | Comedy     | 1983-12-20 | 747 |
| Anne Hathaway     | P292630         | Drama      | 1982-11-12 | 744 |
| Jennifer Hudson   | P454405         | Drama      | 1981-09-12 | 742 |
| Rinko Kikuchi     | P475244         | Drama      | 1981-01-06 | 739 |

---

**oscar\_nominees**

| year | category                     | nominee           | movie                   | winner | id   |
| ---- | ---------------------------- | ----------------- | ----------------------- | ------ | ---- |
| 2008 | actress in a leading role    | Anne Hathaway     | Rachel Getting Married  | 0      | 77   |
| 2012 | actress in a supporting role | Anne HathawayLes  | Mis\_rables             | 1      | 78   |
| 2006 | actress in a supporting role | Jennifer Hudson   | Dreamgirls              | 1      | 711  |
| 2010 | actress in a leading role    | Jennifer Lawrence | Winters Bone            | 1      | 717  |
| 2012 | actress in a leading role    | Jennifer Lawrence | Silver Linings Playbook | 1      | 718  |
| 2011 | actor in a supporting role   | Jonah Hill        | Moneyball               | 0      | 799  |
| 2006 | actress in a supporting role | Rinko Kikuchi     | Babel                   | 0      | 1253 |

---

### Explanation

1. **WinnerCount CTE**:
   Calculate the total Oscar wins per nominee by counting rows with `winner = 1`.

2. **Final Selection**:

   * Join the winner counts with `nominee_information` on nominee name.
   * Order by `total_wins` descending and `name` ascending.
   * Return the top result’s `top_genre`.

---



## Q20. Tesla (Medium Level)

---

**You are given a table of product launches by company by year. Write a query to count the net difference between the number of products companies launched in 2020 with the number of products companies launched in the previous year. Output the name of the companies and a net difference of net products released for 2020 compared to the previous year.**

---

### Schema and Dataset

**car\_launches**

| year | company\_name | product\_name |
| ---- | ------------- | ------------- |
| 2019 | Toyota        | Avalon        |
| 2019 | Toyota        | Camry         |
| 2020 | Toyota        | Corolla       |
| 2019 | Honda         | Accord        |
| 2019 | Honda         | Passport      |
| 2019 | Honda         | CR-V          |
| 2020 | Honda         | Pilot         |
| 2019 | Honda         | Civic         |
| 2020 | Chevrolet     | Trailblazer   |
| 2020 | Chevrolet     | Trax          |
| 2019 | Chevrolet     | Traverse      |
| 2020 | Chevrolet     | Blazer        |
| 2019 | Ford          | Figo          |
| 2020 | Ford          | Aspire        |
| 2019 | Ford          | Endeavour     |
| 2020 | Jeep          | Wrangler      |

---

### Explanation to Solve Query

1. **Counting Products per Year:**
   Use `SUM` with `CASE` statements to count how many products each company launched in 2020 and in 2019.

2. **Calculating Net Difference:**
   Compute the difference between product counts in 2020 and 2019 for each company to get the net change.

3. **Ordering:**
   Order the results by `net_difference` descending so companies with the biggest increase appear first.

---



## Q21. IBM (Hard Level)

---

**IBM is working on a new feature to analyze user purchasing behavior for all Fridays in the first quarter of the year. For each Friday separately, calculate the average amount users have spent per order. The output should contain the week number of that Friday and average amount spent.**

---

### Schema and Dataset

**user\_purchases**

| user\_id | date       | amount\_spent | day\_name |
| -------- | ---------- | ------------- | --------- |
| 1047     | 2023-01-01 | 288           | Sunday    |
| 1099     | 2023-01-04 | 803           | Wednesday |
| 1055     | 2023-01-07 | 546           | Saturday  |
| 1040     | 2023-01-10 | 680           | Tuesday   |
| 1052     | 2023-01-13 | 889           | Friday    |
| 1052     | 2023-01-13 | 596           | Friday    |
| 1016     | 2023-01-16 | 960           | Monday    |
| 1023     | 2023-01-17 | 861           | Tuesday   |
| 1010     | 2023-01-19 | 758           | Thursday  |
| 1013     | 2023-01-19 | 346           | Thursday  |
| 1069     | 2023-01-21 | 541           | Saturday  |
| 1030     | 2023-01-22 | 175           | Sunday    |
| 1034     | 2023-01-23 | 707           | Monday    |
| 1019     | 2023-01-25 | 253           | Wednesday |
| 1052     | 2023-01-25 | 868           | Wednesday |
| 1095     | 2023-01-27 | 424           | Friday    |
| 1017     | 2023-01-28 | 755           | Saturday  |
| 1010     | 2023-01-29 | 615           | Sunday    |
| 1063     | 2023-01-31 | 534           | Tuesday   |
| 1019     | 2023-02-03 | 185           | Friday    |
| 1019     | 2023-02-03 | 995           | Friday    |
| 1092     | 2023-02-06 | 796           | Monday    |
| 1058     | 2023-02-09 | 384           | Thursday  |
| 1055     | 2023-02-12 | 319           | Sunday    |
| 1090     | 2023-02-15 | 168           | Wednesday |
| 1090     | 2023-02-18 | 146           | Saturday  |
| 1062     | 2023-02-21 | 193           | Tuesday   |
| 1023     | 2023-02-24 | 259           | Friday    |

---

### Explanation to Solve Query

1. Identify the Fridays in the first quarter (Q1) of the year.

2. Calculate the week number for each Friday.

3. Group purchases by week number and calculate the average amount spent per order.

---



## Q22. Microsoft (Medium Level)

---

**Given a list of projects and employees mapped to each project, calculate by the amount of project budget allocated to each employee. The output should include the project title and the project budget rounded to the closest integer. Order your list by projects with the highest budget per employee first.**

---

### Schema and Dataset

**ms\_projects**

| id | title     | budget |
| -- | --------- | ------ |
| 1  | Project1  | 29498  |
| 2  | Project2  | 32487  |
| 3  | Project3  | 43909  |
| 4  | Project4  | 15776  |
| 5  | Project5  | 36268  |
| 6  | Project6  | 41611  |
| 7  | Project7  | 34003  |
| 8  | Project8  | 49284  |
| 9  | Project9  | 32341  |
| 10 | Project10 | 47587  |
| 11 | Project11 | 11705  |
| 12 | Project12 | 10468  |
| 13 | Project13 | 43238  |
| 14 | Project14 | 30014  |
| 15 | Project15 | 48116  |
| 16 | Project16 | 19922  |
| 17 | Project17 | 19061  |
| 18 | Project18 | 10302  |
| 19 | Project19 | 44986  |
| 20 | Project20 | 19497  |

**ms\_emp\_projects**

| emp\_id | project\_id |
| ------- | ----------- |
| 10592   | 1           |
| 10593   | 2           |
| 10594   | 3           |
| 10595   | 4           |
| 10596   | 5           |
| 10597   | 6           |
| 10598   | 7           |
| 10599   | 8           |
| 10600   | 9           |
| 10601   | 10          |
| 10602   | 11          |
| 10603   | 12          |
| 10604   | 13          |
| 10605   | 14          |
| 10606   | 15          |
| 10607   | 16          |
| 10608   | 17          |
| 10609   | 18          |
| 10610   | 19          |
| 10611   | 20          |

---

### Explanation of the Query

1. **Joining Tables:** The initial step involves joining the `ms_projects` and `ms_emp_projects` tables on the project ID to combine project details (including titles and budgets) with employee assignments.

2. **Grouping and Aggregating:** The data is then grouped by project title and budget, allowing for the calculation of budget per employee by dividing the total budget of each project by the count of employees assigned to that project.

3. **Rounding and Ordering:** Finally, the computed budget per employee is rounded to the nearest integer, and the results are ordered in descending order to prioritize projects with the highest budget allocation per employee.

---



## Q23. Airbnb (Medium Level)

---

**Find the total number of available beds per hosts' nationality.
Output the nationality along with the corresponding total number of available beds.
Sort records by the total available beds in descending order.**

---

### Schema and Dataset

**airbnb\_apartments**

| host\_id | apartment\_id | apartment\_type | n\_beds | n\_bedrooms | country | city      |
| -------- | ------------- | --------------- | ------- | ----------- | ------- | --------- |
| 0        | A1            | Room            | 1       | 1           | USA     | NewYork   |
| 0        | A2            | Room            | 1       | 1           | USA     | NewJersey |
| 0        | A3            | Room            | 1       | 1           | USA     | NewJersey |
| 1        | A4            | Apartment       | 2       | 1           | USA     | Houston   |
| 1        | A5            | Apartment       | 2       | 1           | USA     | LasVegas  |
| 3        | A7            | Penthouse       | 3       | 3           | China   | Tianjin   |
| 3        | A8            | Penthouse       | 5       | 5           | China   | Beijing   |
| 4        | A9            | Apartment       | 2       | 1           | Mali    | Bamako    |
| 5        | A10           | Room            | 3       | 1           | Mali    | Segou     |

**airbnb\_hosts**

| host\_id | nationality | gender | age |
| -------- | ----------- | ------ | --- |
| 0        | USA         | M      | 28  |
| 1        | USA         | F      | 29  |
| 2        | China       | F      | 31  |
| 3        | China       | M      | 24  |
| 4        | Mali        | M      | 30  |
| 5        | Mali        | F      | 30  |

---

### Explanation of the Query

1. **Joining Tables:** The first step involves joining the `airbnb_apartments` and `airbnb_hosts` tables on the `host_id`. This allows us to combine apartment details (like `n_beds`) with the host's nationality.

2. **Grouping and Aggregating:** We group the combined data by nationality to calculate the total number of available beds (`SUM(n_beds)`) per nationality.

3. **Sorting the Results:** The final results are sorted in descending order by the total number of beds so that nationalities with the most available beds appear at the top.

---

Certainly! Here's your formatted entry with a sequence number and structured just like your previous ones:

---

## Q24. Uber (Hard Level)

---

**Some forecasting methods are extremely simple and surprisingly effective. Naïve forecast is one of them. To create a naïve forecast for "distance per dollar" (defined as `distance_to_travel / monetary_cost`), first sum the "distance to travel" and "monetary cost" values monthly. This gives the actual value for the current month. For the forecasted value, use the previous month's value.**

**After obtaining both actual and forecasted values, calculate the root mean squared error (RMSE) using the formula:**

> `RMSE = sqrt(mean(square(actual - forecast)))`

**Report the RMSE rounded to two decimal places.**

---

### Schema and Dataset

**uber\_request\_logs**

| request\_id | request\_date       | request\_status | distance\_to\_travel | monetary\_cost | driver\_to\_client\_distance |
| ----------- | ------------------- | --------------- | -------------------- | -------------- | ---------------------------- |
| 1           | 2020-01-09 00:00:00 | success         | 70.59                | 6.56           | 14.36                        |
| 2           | 2020-01-24 00:00:00 | success         | 93.36                | 22.68          | 19.9                         |
| 3           | 2020-02-08 00:00:00 | fail            | 51.24                | 11.39          | 21.32                        |
| 4           | 2020-02-23 00:00:00 | success         | 61.58                | 8.04           | 44.26                        |
| 5           | 2020-03-09 00:00:00 | success         | 25.04                | 7.19           | 1.74                         |
| 6           | 2020-03-24 00:00:00 | fail            | 45.57                | 4.68           | 24.19                        |
| 7           | 2020-04-08 00:00:00 | success         | 24.45                | 12.69          | 15.91                        |
| 8           | 2020-04-23 00:00:00 | success         | 48.22                | 11.2           | 48.82                        |
| 9           | 2020-05-08 00:00:00 | success         | 56.63                | 4.04           | 16.08                        |
| 10          | 2020-05-23 00:00:00 | fail            | 19.03                | 16.65          | 11.22                        |
| 11          | 2020-06-07 00:00:00 | fail            | 81                   | 6.56           | 26.6                         |
| 12          | 2020-06-22 00:00:00 | fail            | 21.32                | 8.86           | 28.57                        |
| 13          | 2020-07-07 00:00:00 | fail            | 14.74                | 17.76          | 19.33                        |
| 14          | 2020-07-22 00:00:00 | success         | 66.73                | 13.68          | 14.07                        |
| 15          | 2020-08-06 00:00:00 | success         | 32.98                | 16.17          | 25.34                        |
| 16          | 2020-08-21 00:00:00 | success         | 46.49                | 1.84           | 41.9                         |
| 17          | 2020-09-05 00:00:00 | fail            | 45.98                | 12.2           | 2.46                         |
| 18          | 2020-09-20 00:00:00 | success         | 3.14                 | 24.8           | 36.6                         |
| 19          | 2020-10-05 00:00:00 | success         | 75.33                | 23.04          | 29.99                        |
| 20          | 2020-10-20 00:00:00 | success         | 53.76                | 22.94          | 18.74                        |

---

### Explanation of the Query

1. **`monthly_aggregates` CTE**:
   Aggregates the data by extracting year and month from `request_date`. It sums `distance_to_travel` and `monetary_cost` as `total_distance` and `total_cost`.

2. **`distance_per_dollar` CTE**:
   Calculates the monthly metric `distance_per_dollar = total_distance / total_cost`.

3. **`naive_forecast` CTE**:
   Uses the `LAG()` window function to get the previous month’s `distance_per_dollar` as the forecast for the current month.

4. **Final Select Statement**:
   Filters out the first month (as it has no prior month), computes the squared error between actual and forecast values, and then calculates the RMSE using `SQRT(AVG(...))`.
   The final result is rounded to **2 decimal places**.

---


## Q25. Google (Medium Level)

---

**You are analyzing a social network dataset at Google. Your task is to find mutual friends between two users, *Karl* and *Hans*.**

> *There is only one user named Karl and one named Hans in the dataset.*

---

### ✨ Output should contain:

* `user_id`
* `user_name` (of the mutual friends)

---

### 🔗 Understanding how to join tables in SQL is essential for effective data analysis; mastering this concept allows you to combine related data seamlessly. Give it a try! 👇

---

### Schema and Dataset

**`users` Table**

| user\_id | user\_name |
| -------- | ---------- |
| 1        | Karl       |
| 2        | Hans       |
| 3        | Emma       |
| 4        | Emma       |
| 5        | Mike       |
| 6        | Lucas      |
| 7        | Sarah      |
| 8        | Lucas      |
| 9        | Anna       |
| 10       | John       |

**`friends` Table**

| user\_id | friend\_id |
| -------- | ---------- |
| 1        | 3          |
| 1        | 5          |
| 2        | 3          |
| 2        | 4          |
| 3        | 1          |
| 3        | 2          |
| 3        | 6          |
| 4        | 7          |
| 5        | 8          |
| 6        | 9          |
| 7        | 10         |
| 8        | 6          |
| 9        | 10         |
| 10       | 7          |
| 10       | 9          |

---

### 🧠 Explanation of the Query:

1. **CTE `karl_friends` and `hans_friends`**:
   Retrieve all friends for Karl (`user_id = 1`) and Hans (`user_id = 2`) from the `friends` table.

2. **Main Query**:
   Finds the intersection (mutual friends) between Karl’s and Hans’s friends using an `INNER JOIN` on `friend_id`.

3. **Final Join with `users` Table**:
   Retrieves the `user_name` and `user_id` of the mutual friends using a `JOIN` with the `users` table.

---


## Q26. Amazon (Hard Level)

---

**Given a table `sf_transactions` of purchases by date, calculate the month-over-month percentage change in revenue.**

> *Output should include the year-month date (YYYY-MM) and percentage change, rounded to two decimal places.*

---

###  Output Columns:

* `year_month` (formatted as `YYYY-MM`)
* `percentage_change` (rounded to 2 decimal places)

> The percentage change is calculated as:
> `((this month’s revenue - last month’s revenue) / last month’s revenue) * 100`
> *Note: First month will have `NULL` since there’s no prior month to compare.*

---

###  Schema and Dataset

**`sf_transactions` Table**

| id | created\_at         | value  | purchase\_id |
| -- | ------------------- | ------ | ------------ |
| 1  | 2019-01-01 00:00:00 | 172692 | 43           |
| 2  | 2019-01-05 00:00:00 | 177194 | 36           |
| 3  | 2019-01-09 00:00:00 | 109513 | 30           |
| 4  | 2019-01-13 00:00:00 | 164911 | 30           |
| 5  | 2019-01-17 00:00:00 | 198872 | 39           |
| 6  | 2019-01-21 00:00:00 | 184853 | 31           |
| 7  | 2019-01-25 00:00:00 | 186817 | 26           |
| 8  | 2019-01-29 00:00:00 | 137784 | 22           |
| 9  | 2019-02-02 00:00:00 | 140032 | 25           |
| 10 | 2019-02-06 00:00:00 | 116948 | 43           |
| 11 | 2019-02-10 00:00:00 | 162515 | 25           |
| 12 | 2019-02-14 00:00:00 | 114256 | 12           |
| 13 | 2019-02-18 00:00:00 | 197465 | 48           |
| 14 | 2019-02-22 00:00:00 | 120741 | 20           |
| 15 | 2019-02-26 00:00:00 | 100074 | 49           |
| 16 | 2019-03-02 00:00:00 | 157548 | 19           |
| 17 | 2019-03-06 00:00:00 | 105506 | 16           |
| 18 | 2019-03-10 00:00:00 | 189351 | 46           |
| 19 | 2019-03-14 00:00:00 | 191231 | 29           |
| 20 | 2019-03-18 00:00:00 | 120575 | 44           |
| 21 | 2019-03-22 00:00:00 | 151688 | 47           |
| 22 | 2019-03-26 00:00:00 | 102327 | 18           |
| 23 | 2019-03-30 00:00:00 | 156147 | 25           |

---

###  Explanation of the Query:

1. **CTE `monthly_revenue`**
   Groups data by month (`FORMAT(created_at, 'yyyy-MM')`) and computes `SUM(value)` as total monthly revenue.

2. **CTE `revenue_with_lag`**
   Uses `LAG()` to bring in previous month’s revenue for comparison.

3. **Final SELECT**
   Calculates percent change:
   `((current - previous) / previous) * 100`, rounded with `ROUND()`.

---
